In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip RaceSQI

In [ ]:
!ls

drive  sample_data


In [ ]:
!fairfaceequ ls

/bin/bash: fairfaceequ: command not found


In [ ]:
cd drive/My\ Drive

/content/drive/My Drive


In [ ]:
ls RaceSQI/Southeast/Female

In [ ]:
!unzip fairfacesqi.zip

Streaming output truncated to the last 5000 lines.
  inflating: __MACOSX/fairfacesqi/Validation/Female/._7748.jpg  
  inflating: fairfacesqi/Validation/Female/9003.jpg  
  inflating: __MACOSX/fairfacesqi/Validation/Female/._9003.jpg  
  inflating: fairfacesqi/Validation/Female/3248.jpg  
  inflating: __MACOSX/fairfacesqi/Validation/Female/._3248.jpg  
  inflating: fairfacesqi/Validation/Female/6318.jpg  
  inflating: __MACOSX/fairfacesqi/Validation/Female/._6318.jpg  
  inflating: fairfacesqi/Validation/Female/10439.jpg  
  inflating: __MACOSX/fairfacesqi/Validation/Female/._10439.jpg  
  inflating: fairfacesqi/Validation/Female/3260.jpg  
  inflating: __MACOSX/fairfacesqi/Validation/Female/._3260.jpg  
  inflating: fairfacesqi/Validation/Female/4269.jpg  
  inflating: __MACOSX/fairfacesqi/Validation/Female/._4269.jpg  
  inflating: fairfacesqi/Validation/Female/3506.jpg  
  inflating: __MACOSX/fairfacesqi/Validation/Female/._3506.jpg  
  inflating: fairfacesqi/Validation/Female/5177.j

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import time

import os
import random
import numpy as np

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
train_transforms = transforms.Compose([
                           transforms.RandomHorizontalFlip(),
                           transforms.RandomRotation(10),
                           transforms.RandomCrop((224, 224), pad_if_needed=True),
                           transforms.ToTensor(),
                           transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
                       ])

test_transforms = transforms.Compose([
                           transforms.CenterCrop((224, 224)),
                           transforms.ToTensor(),
                           transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
                       ])

In [ ]:
train_data = datasets.ImageFolder('fairfacesqi/Training', train_transforms)
valid_data = datasets.ImageFolder('fairfacesqi/Validation',test_transforms)
#test_data = datasets.ImageFolder('RaceSQI/', test_transforms)


In [ ]:
print(valid_data.class_to_idx)

{'Female': 0, 'Male': 1}


https://github.com/facebook/fb.resnet.torch/issues/180
https://github.com/bamos/densenet.pytorch/blob/master/compute-cifar10-mean.py

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')


Number of training examples: 85902
Number of validation examples: 10836


In [ ]:
BATCH_SIZE = 64

train_iterator = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
valid_iterator = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE)


https://discuss.pytorch.org/t/why-does-the-resnet-model-given-by-pytorch-omit-biases-from-the-convolutional-layer/10990/4
https://github.com/kuangliu/pytorch-cifar/blob/master/models/resnet.py

In [ ]:
device = torch.device('cuda')

In [ ]:
import torchvision.models as models

model = models.resnet18(pretrained=True).to(device)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
print(model)

In [ ]:
from torch.optim import lr_scheduler


num_ftrs = model.fc.in_features
model.fc = nn.Linear(512, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

Adam optimizer

In [ ]:
from torch.optim import lr_scheduler


num_ftrs = model.fc.in_features
model.fc = nn.Linear(512, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999),eps=1e-08, weight_decay=0, amsgrad=False)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
print(exp_lr_scheduler.state_dict())

{'step_size': 7, 'gamma': 0.1, 'base_lrs': [0.0001], 'last_epoch': 0, '_step_count': 1, 'verbose': False, '_get_lr_called_within_step': False, '_last_lr': [0.0001]}


In [ ]:
def calculate_accuracy(fx, y):
    preds = fx.max(1, keepdim=True)[1]
    correct = preds.eq(y.view_as(preds)).sum()
    acc = correct.float()/preds.shape[0]
    return acc

In [ ]:
def train(model, device, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        fx = model(x)
        
        loss = criterion(fx, y)
        
        acc = calculate_accuracy(fx, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, device, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            fx = model(x)

            loss = criterion(fx, y)

            acc = calculate_accuracy(fx, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
EPOCHS = 60
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'ColabRaceSQI-ResentFT-males-vs-females.pt')

best_valid_loss = float('inf')

if not os.path.isdir(f'{SAVE_DIR}'):
    os.makedirs(f'{SAVE_DIR}')
history = []
for epoch in range(EPOCHS):
    epoch_start = time.time()
    train_loss, train_acc = train(model, device, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, device, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)

    history.append([train_loss, valid_loss, train_acc, valid_acc])
    epoch_end = time.time()
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:05.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:05.2f}% | Time: {epoch_end-epoch_start:.4f}s |')

| Epoch: 01 | Train Loss: 0.567 | Train Acc: 69.43% | Val. Loss: 0.467 | Val. Acc: 76.48% | Time: 1018.6532s |
| Epoch: 02 | Train Loss: 0.498 | Train Acc: 74.41% | Val. Loss: 0.436 | Val. Acc: 78.26% | Time: 942.3121s |
| Epoch: 03 | Train Loss: 0.471 | Train Acc: 76.02% | Val. Loss: 0.411 | Val. Acc: 79.96% | Time: 910.5878s |
| Epoch: 04 | Train Loss: 0.459 | Train Acc: 76.73% | Val. Loss: 0.440 | Val. Acc: 79.46% | Time: 907.0585s |
| Epoch: 05 | Train Loss: 0.446 | Train Acc: 77.64% | Val. Loss: 0.388 | Val. Acc: 81.08% | Time: 911.0607s |
| Epoch: 06 | Train Loss: 0.436 | Train Acc: 78.42% | Val. Loss: 0.382 | Val. Acc: 81.41% | Time: 926.8882s |
| Epoch: 07 | Train Loss: 0.429 | Train Acc: 78.74% | Val. Loss: 0.384 | Val. Acc: 80.85% | Time: 934.5248s |
| Epoch: 08 | Train Loss: 0.420 | Train Acc: 79.15% | Val. Loss: 0.369 | Val. Acc: 82.36% | Time: 966.8210s |
| Epoch: 09 | Train Loss: 0.414 | Train Acc: 79.58% | Val. Loss: 0.356 | Val. Acc: 82.78% | Time: 930.6975s |
| Epoch: 

In [ ]:
EPOCHS = 100
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'ClbSQIRace-ResentFT-males-vs-females.pt')

best_valid_loss = float('inf')

if not os.path.isdir(f'{SAVE_DIR}'):
    os.makedirs(f'{SAVE_DIR}')
history = []
for epoch in range(EPOCHS):
    epoch_start = time.time()
    train_loss, train_acc = train(model, device, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, device, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)

    history.append([train_loss, valid_loss, train_acc, valid_acc])
    epoch_end = time.time()
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:05.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:05.2f}% | Time: {epoch_end-epoch_start:.4f}s |')

| Epoch: 01 | Train Loss: 0.568 | Train Acc: 69.04% | Val. Loss: 0.471 | Val. Acc: 76.64% | Time: 1002.2489s |
| Epoch: 02 | Train Loss: 0.497 | Train Acc: 74.33% | Val. Loss: 0.426 | Val. Acc: 78.96% | Time: 963.0874s |
| Epoch: 03 | Train Loss: 0.473 | Train Acc: 76.06% | Val. Loss: 0.407 | Val. Acc: 80.13% | Time: 928.4486s |
| Epoch: 04 | Train Loss: 0.457 | Train Acc: 76.93% | Val. Loss: 0.405 | Val. Acc: 80.57% | Time: 937.6829s |
| Epoch: 05 | Train Loss: 0.445 | Train Acc: 77.44% | Val. Loss: 0.401 | Val. Acc: 80.37% | Time: 938.2751s |
| Epoch: 06 | Train Loss: 0.439 | Train Acc: 78.07% | Val. Loss: 0.407 | Val. Acc: 79.80% | Time: 932.6804s |
| Epoch: 07 | Train Loss: 0.426 | Train Acc: 78.89% | Val. Loss: 0.368 | Val. Acc: 82.15% | Time: 979.0600s |
| Epoch: 08 | Train Loss: 0.420 | Train Acc: 79.31% | Val. Loss: 0.366 | Val. Acc: 82.01% | Time: 943.4135s |
| Epoch: 09 | Train Loss: 0.413 | Train Acc: 79.75% | Val. Loss: 0.377 | Val. Acc: 82.09% | Time: 946.5183s |
| Epoch: 

In [ ]:
#valid_data = datasets.ImageFolder('RaceSQI/', test_transforms)
test_data = datasets.ImageFolder('RaceSQI/', test_transforms)


test_iterator = torch.utils.data.DataLoader(test_data, batch_size=64)
#valid_iterator = torch.utils.data.DataLoader(valid_data, batch_size=64)
model.load_state_dict(torch.load('models/SQI-ResentFT-males-vs-females.pt'))

test_loss, test_acc = evaluate(model, device, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:05.2f}% |')

In [ ]:
import matplotlib.pyplot as plt
history = np.array(history)
plt.plot(history[:,0:2])
plt.legend(['Tr Loss', 'Val Loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.ylim(0,10)
plt.savefig('loss_curve.png')
plt.show()

NameError: ignored

In [ ]:
plt.plot(history[:,2:4])
plt.legend(['Tr Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.ylim(0,1)
plt.savefig('accuracy_curve.png')
plt.show()

In [ ]:
print(optimizer.state_dict())

f = open( 'test_optimizer_state_dict.txt', 'w' )
f.write( 'test_optimizer_state_dict = ' + repr(optimizer.state_dict()) + '\n' )
f.close()

{'state': {}, 'param_groups': [{'lr': 0.0001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'initial_lr': 0.0001, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]}]}


In [ ]:
#print(model.state_dict())

f = open( 'test_model_state_dict.txt', 'w' )
f.write( 'test_model_state_dict = ' + repr(model.state_dict()) + '\n' )
f.close()

In [ ]:
dict = {'one': 1, 'two': 2}
f = open( 'test123.txt', 'w' )
f.write( 'dict = ' + repr(dict) + '\n' )
f.close()

In [ ]:
print(history[:,3:4])
flat_list = [item for sublist in history[:,3:4] for item in sublist]

In [ ]:
best_acc=max(flat_list)
print(best_acc)

In [ ]:
#trial1 = torch.load('epoch_60.pt')
#print(trial1)
#print(model)
import shutil
def save_checkpoint123(state123, is_best123, filename123='checkpoint.pth.tar'):
    torch.save(state123, filename123)
    if is_best123:
        shutil.copyfile(filename123, 'model_best.pth.tar')
        
save_checkpoint123({
            'epoch': epoch + 1,
            'arch': 'resnet18',
            'state_dict': model.state_dict(),
            'best_prec1': best_acc,
            'optimizer' : optimizer.state_dict(),
            'all_history': history,
            'exp_lr_scheduler_dict' : exp_lr_scheduler.state_dict(),
        }, True)

In [ ]:
resume = 'model_best.pth.tar'
if resume:
        if os.path.isfile(resume):
            print("=> loading checkpoint '{}'".format(resume))
            checkpoint = torch.load(resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            all_history = (checkpoint['all_history']).tolist()
            exp_lr_scheduler = checkpoint['exp_lr_scheduler_dict']
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

In [ ]:
print(start_epoch)
print(all_history)
history = all_history

26
[[0.4877623362676119, 0.42979410168002635, 0.749431576466081, 0.7883272058823529], [0.4675564170549454, 0.43807121068239213, 0.7627130757493042, 0.7841176471289466], [0.4512634763323542, 0.41692360727226035, 0.7706527231526038, 0.7959374999298768], [0.4405219006343112, 0.37915745912229315, 0.7783098207954497, 0.8165073531515458], [0.43248980316490887, 0.37413848156438156, 0.7849181603337968, 0.8214889705181122], [0.42353124835556866, 0.4005695945199798, 0.7881408892988004, 0.803823529622134], [0.41850385268455287, 0.3847169643815826, 0.7923242873605826, 0.8112316177171819], [0.4113658732406612, 0.3622644422685399, 0.7972639214167023, 0.8248161764705882], [0.4072571987992157, 0.36078711476396114, 0.8007160142918442, 0.8299264707986046], [0.4046311090774529, 0.3552451130221872, 0.8013475960257864, 0.8341176471289466], [0.3976249680400161, 0.42641238289720873, 0.8050905489158204, 0.8045404413167168], [0.3938840890964819, 0.37251593943904426, 0.8058185299563035, 0.8224448530112995], [0.

In [ ]:
EPOCHS = 1
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'EpochIncrementRaceSQI-ResentFT-males-vs-females.pt')

best_valid_loss = float('inf')

if not os.path.isdir(f'{SAVE_DIR}'):
    os.makedirs(f'{SAVE_DIR}')
history = []
for epoch in range(start_epoch,start_epoch+EPOCHS):
    epoch_start = time.time()
    train_loss, train_acc = train(model, device, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, device, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
    
    history.append([train_loss, valid_loss, train_acc, valid_acc])
    epoch_end = time.time()
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:05.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:05.2f}% | Time: {epoch_end-epoch_start:.4f}s |')
    

In [ ]:
#model.load_state_dict(torch.load('models/EpochIncrementRaceSQI-ResentFT-males-vs-females.pt'))
valid_data = datasets.ImageFolder('fairfacesqi/Validation', test_transforms)
#test_data = datasets.ImageFolder(dataset+'\\validation', test_transforms)
class_names = valid_data.classes

dataloaders = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE)
nb_classes = 460


confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
print(confusion_matrix)
print(confusion_matrix.diag()/confusion_matrix.sum(1))
Ind_acc = confusion_matrix.diag()/confusion_matrix.sum(1)
print("Female : ", float(Ind_acc[0]))
print("Male : ", float(Ind_acc[1]))
TN = int(confusion_matrix[0][0])
FN = int(confusion_matrix[0][1])
FP = int(confusion_matrix[1][0])
TP = int(confusion_matrix[1][1])

In [ ]:
FPR = 0
FNR = 0
TPR = 0
TNR = 0
FPR = FP/(FP + TN)
FNR = FN/(FN + TP)
TPR = 1 - FNR
TNR = 1 - FPR
Overall_acc = (TPR + TNR)/2
print("Overall Accuracy : ", Overall_acc)
print("Female : ", float(Ind_acc[0]))
print("Male : ", float(Ind_acc[1]))
print("FPR : ", FNR)
print("FNR : ", FPR)

Overall Accuracy :  0.8187699653955639
Female :  0.8097570538520813
Male :  0.8279832601547241
FPR :  0.16984432394612559
FNR :  0.19261574526274663


In [ ]:
    model.eval()
    num_correct = 0 
    num_examples = 0
    for batch in val_loader:
        inputs, targets = batch
        inputs = inputs.to(device)
        output = model(inputs)
        targets = targets.to(device)
        loss = loss_fn(output,targets) 
        valid_loss += loss.data.item() * inputs.size(0)
        correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
        num_correct += torch.sum(correct).item()
        num_examples += correct.shape[0]
    valid_loss /= len(val_loader.dataset)
 
    print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
    valid_loss, num_correct / num_examples))

In [ ]:
summary(model)


In [ ]:

from torchsummary import summary

end of epoch increment

In [ ]:
LL

In [ ]:
LL

In [ ]:
ll

In [ ]:
!unzip RaceSQI.zip

unzip:  cannot find or open RaceSQI.zip, RaceSQI.zip.zip or RaceSQI.zip.ZIP.


In [ ]:
import os
#model.load_state_dict(torch.load('models/EpochTest-1-RaceSQI-ResentFT-males-vs-females.pt'))
#val = r'../pradeep/Desktop/FinalRacedataets/RaceEqu/Validation/'
val = 'RaceSQI/'

races = os.listdir(val)
genders = ['Male','Female']
for race in races:
    for gender in genders:
        finalPath = val
        #print(finalPath)


        #test_data = datasets.ImageFolder(finalPath, test_transforms)
        valid_data = datasets.ImageFolder(finalPath, test_transforms)

        #test_iterator = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)
        valid_iterator = torch.utils.data.DataLoader(valid_data, batch_size=64)
        model.load_state_dict(torch.load('models/EpochIncrementRaceSQI-ResentFT-males-vs-females.pt'))

        test_loss, test_acc = evaluate(model, device, valid_iterator,criterion)

        print(f'{race} {gender}| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:05.2f}% |')





In [ ]:
#val = r'../pradeep/Desktop/FinalRacedataets/RaceEqu/Validation/'
val = 'RaceSQI/'
races = os.listdir(val)
genders = ['Male','Female']
for race in races:
    for gender in genders:
        finalPath = val+race+'/'+gender
        print(finalPath)


        test_data = datasets.ImageFolder(finalPath, test_transforms)


        test_iterator = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)
        model.load_state_dict(torch.load('models/ResentFT-males-vs-females.pt'))

        test_loss, test_acc = evaluate(model, device, test_iterator, criterion)

        print(f'{race} {gender}| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:05.2f}% |')


In [ ]:
data_transforms = {
    'test': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'dataset'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['test', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['6-10', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['6-10', 'val']}
class_names = image_datasets['val'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model.load_state_dict(torch.load('models/EpochIncrementRaceSQI-ResentFT-males-vs-females.pt'))
test_data = datasets.ImageFolder('fairfaceequ/Validation', test_transforms)
class_names = test_data.classes

dataloaders = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)
nb_classes = 2


confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
print(confusion_matrix.diag()/confusion_matrix.sum(1))

tensor([0.8932, 0.9009])


In [ ]:
ls RaceSQI

ls: cannot access 'RaceSQI': No such file or directory


In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion() 

def visualize_model(model, num_images=22):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
           # print('GroundTruth: ', ' '.join('%5s' % class_names[labels[j]] for j in range(22)))

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                print('GroudthTruth: ',class_names[labels[j]])
                ax.set_title('Predicted: {}'.format(class_names[preds[j]]))
               
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
out = torchvision.utils.make_grid(inputs)

visualize_model(model)
plt.ioff()
plt.show()